In [ ]:
%%html
<div>
  <p>base64 image of a red dot:</p>
  <img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg==" alt="Red dot" />
</div>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = 21
x = np.linspace(0, 10, 11)
y = [3.9, 4.4, 10.8, 10.3, 11.2, 13.1, 14.1,  9.9, 13.9, 15.1, 12.5]

# fit a linear curve and estimate its y-values and their error.
a, b = np.polyfit(x, y, deg=1)
y_est = a * x + b
y_err = x.std() * np.sqrt(1/len(x) +
                          (x - x.mean())**2 / np.sum((x - x.mean())**2))

fig, ax = plt.subplots()
fig.patch.set_facecolor('lightblue')

ax.plot(x, y_est, '-')
ax.fill_between(x, y_est - y_err, y_est + y_err, alpha=0.2)
ax.plot(x, y, 'o', color='tab:brown');

In [ ]:
import io
import base64
from IPython.display import HTML

# TODO - this has to be refactored, this part was mostly generated by chat-gpt.

buf = io.BytesIO()
fig.savefig(buf, format='png')

buf.seek(0)
base64_img_string_pre = base64.b64encode(buf.getvalue()).decode()


base64_img_string = f"data:image/png;base64,{base64_img_string_pre}"

# base64 string of the PNG image
# b64_string = 'iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg==' # for debugging
b64_string = base64_img_string_pre

# decode base64 string to bytes
decoded_bytes = base64.b64decode(b64_string)

# check if the file has the PNG signature
if decoded_bytes[:8] != b'\x89PNG\r\n\x1a\n':
    raise ValueError('Invalid PNG file')

# extract the IHDR chunk
ihdr_start = 8
ihdr_end = decoded_bytes.find(b'IHDR') + 4 + 8
ihdr_chunk = decoded_bytes[ihdr_start:ihdr_end]

# extract image width and height from the IHDR chunk
image_width = int.from_bytes(ihdr_chunk[8:12], byteorder='big')
image_height = int.from_bytes(ihdr_chunk[12:16], byteorder='big')

display(HTML(f'<img src={base64_img_string}>'))
print(image_width, image_height)

In [ ]:
import ipyreact
from traitlets import Unicode, Int

class Tldraw(ipyreact.ReactWidget):
    image_width = Int(300).tag(sync=True)
    image_height = Int(100).tag(sync=True)
    base64img = Unicode("").tag(sync=True)

    _esm = """
    import * as React from "react";
    import { TDAssetType, TDShapeType, Tldraw, TldrawApp } from "@tldraw/tldraw";

    export default function App({image_width, image_height, base64img}) {
      const handleMount = (app: TldrawApp) => {
        // You can use the app API here! e.g. app.selectAll()
        app.patchAssets({
          myAssetId: {
            id: "myAssetId",
            type: TDAssetType.Image,
            size: [image_width, image_height],
            fileName: "card-repo.png",
            src: base64img
          }
        });

        app.createShapes({
          id: "myImage",
          type: TDShapeType.Image,
          assetId: "myAssetId",
          point: [0, 60],
          size: [image_width, image_height],
          isLocked: true
        });
      };

    return (
            <div
            style={{
                position: "relative",
                width: image_width,
                height: image_height+ 120
            }}
            >
            <Tldraw onMount={handleMount} showPages={false} showMenu={false} />
            </div>
        )
        }
    """
t = Tldraw(image_width = image_width, image_height = image_height, base64img = base64_img_string)
t